<a href="https://colab.research.google.com/github/gurudattamanpreet/Internship-Projects/blob/main/Arise_Internship_Week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openpyxl as op
import numpy as np
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment
import re
import json
import fitz  # PyMuPDF for PDF
import xml.etree.ElementTree as ET
from docx import Document
import os

In [ ]:
wb=op.Workbook()
sheet=wb.active
sheet.title='Sheet1'
sheet['A1']='Name'
sheet['B1']='Age'
sheet['C1']='City'
sheet.append(['Manpreet','24'])
data = [
    ["Manpreet", 24, "Delhi"],
    ["Rahul", 22, "Mumbai"],
    ["Priya", 25, "Bangalore"],
    ["Aman", 27, "Chennai"],
    ["Neha", 23, "Kolkata"],]
for row in data:
    sheet.append(row)
wb.save("sample.xlsx")
print("Excel file created successfully with multiple records!")

Excel file created successfully with multiple records!


In [ ]:
df=pd.read_excel('/Users/manpreetsingh/Documents/Practise Programms/sample.xlsx')

In [ ]:
df.head(6)

,Name,Age,City
0,Manpreet,24,NaN
1,Manpreet,24,Delhi
2,Rahul,22,Mumbai
3,Priya,25,Bangalore
4,Aman,27,Chennai
5,Neha,23,Kolkata


In [ ]:
sheet['A1'].font = Font(bold=True, color='FF0000')  # Red and Bold
sheet['A1'].alignment = Alignment(horizontal='center')

wb.save('formatted.xlsx')
print('Excel file formatted successfully!')

Excel file formatted successfully!


In [ ]:
df.head(6)

,Name,Age,City
0,Manpreet,24,NaN
1,Manpreet,24,Delhi
2,Rahul,22,Mumbai
3,Priya,25,Bangalore
4,Aman,27,Chennai
5,Neha,23,Kolkata


In [ ]:
df.isna()

,Name,Age,City
0,False,False,True
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False


In [ ]:
df.index.duplicated()

array([False, False, False, False, False, False])

In [ ]:
df=df.dropna()

In [ ]:
df.head()

,Name,Age,City
1,Manpreet,24,Delhi
2,Rahul,22,Mumbai
3,Priya,25,Bangalore
4,Aman,27,Chennai
5,Neha,23,Kolkata


In [ ]:
df.isna()

,Name,Age,City
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False


In [ ]:
import re
import os
import json
import xml.etree.ElementTree as ET
import pandas as pd
import PyPDF2
import docx

def extract_mobile_numbers(text):
    pattern = r'\b[6789]\d{9}\b'
    return re.findall(pattern, text)

def extract_from_pdf(file_path):
    text = ""
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return extract_mobile_numbers(text)

def extract_from_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    text = json.dumps(data)  # JSON ko text me convert kiya
    return extract_mobile_numbers(text)

def extract_from_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    text = ET.tostring(root, encoding="utf-8").decode()
    return extract_mobile_numbers(text)

def extract_from_word(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return extract_mobile_numbers(text)

# Taking input from user
file_path = input("Enter the file path (PDF, JSON, XML, DOCX): ")

# If file already exists
if not os.path.exists(file_path):
    print("Error: File not found!")
    exit()

# File type:
file_ext = file_path.split(".")[-1].lower()

if file_ext == "pdf":
    numbers = extract_from_pdf(file_path)
elif file_ext == "json":
    numbers = extract_from_json(file_path)
elif file_ext == "xml":
    numbers = extract_from_xml(file_path)
elif file_ext == "docx":
    numbers = extract_from_word(file_path)
else:
    print("Error: Unsupported file format! Use PDF, JSON, XML, or DOCX.")
    exit()

# Exporting into excel
if numbers:
    df = pd.DataFrame({"Mobile Numbers": numbers})
    df.to_excel("extracted_numbers.xlsx", index=False)
    print(f"✅ {len(numbers)} mobile numbers extracted and saved to 'extracted_numbers.xlsx'")
else:
    print("❌ No mobile numbers found in the file.")

Enter the file path (PDF, JSON, XML, DOCX):  /Users/manpreetsingh/Library/CloudStorage/GoogleDrive-manpreet.gurudatta@mistmandsaur.org/My Drive/My Mac/pending items/single-turn-data.pdf


✅ 120 mobile numbers extracted and saved to 'extracted_numbers.xlsx'


In [ ]:
def clean_data(df):
    """Basic data cleaning operations"""

    print("\n🔹 Initial Data Summary:")
    print(df.info())

    # 🔹 Step 1: Remove duplicate rows
    df = df.drop_duplicates()
    print("\n✅ Duplicate rows removed!")

    # 🔹 Step 2: Remove null values
    df = df.dropna()
    print("✅ Null values removed!")

    # 🔹 Step 3: Trim extra whitespace from string columns
    str_cols = df.select_dtypes(include=["object"]).columns
    df[str_cols] = df[str_cols].apply(lambda x: x.str.strip())
    print("✅ Extra whitespaces removed!")

    # 🔹 Step 4: Convert numeric columns to proper type
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            pass
    print("✅ Data types corrected where possible!")

    # 🔹 Step 5: Detect outliers using IQR method
    numeric_cols = df.select_dtypes(include=["number"]).columns
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        if not outliers.empty:
            print(f"⚠️ Outliers detected in '{col}', consider handling manually.")

    return df

# 📌 Taking input from user
file_path = input("Enter the file path (CSV or Excel): ")

# ✅ If file already exists
if not os.path.exists(file_path):
    print("❌ Error: File not found!")
    exit()

# ✅ Detecting & loading file type
file_ext = file_path.split(".")[-1].lower()

if file_ext == "csv":
    df = pd.read_csv(file_path)
elif file_ext in ["xls", "xlsx"]:
    df = pd.read_excel(file_path)
else:
    print("❌ Error: Unsupported file format! Use CSV or Excel.")
    exit()

# ✅ Data cleaning process
cleaned_df = clean_data(df)

# ✅ Saving Cleaned data
output_file = "cleaned_data.xlsx"
cleaned_df.to_excel(output_file, index=False)
print(f"\n✅ Data cleaning completed! Cleaned data saved as '{output_file}'.")

Enter the file path (CSV or Excel):  /Users/manpreetsingh/Documents/Practise Programms/extracted_numbers.xlsx



🔹 Initial Data Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Mobile Numbers  120 non-null    int64
dtypes: int64(1)
memory usage: 1.1 KB
None

✅ Duplicate rows removed!
✅ Null values removed!
✅ Extra whitespaces removed!
✅ Data types corrected where possible!

✅ Data cleaning completed! Cleaned data saved as 'cleaned_data.xlsx'.
